In [6]:
import requests
from bs4 import BeautifulSoup
import time 
import random 
import re
from fake_useragent import UserAgent
import datetime
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [7]:
class KanobuArticle:
    def __init__(self):
        self.date=""
        self.time=""
        self.words=""
        self.rubr=""
        self.hashtag=""
        self.author=""
        self.head=""
        self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "words":"'+self.words+'", "rubrics":"'+self.rubr+'", "hashtag":"'+self.hashtag+'", "title":"'
        +self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "hashtag":self.hashtag, \
             "title":self.head, "words":self.words, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

In [8]:
def GetPagesArticles(number_of_pages):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    url = 'https://kanobu.ru/news/?page='
    links = []
    for page in range(1, number_of_pages+1):
        r = requests.get(url + str(page), headers = headers)
        time.sleep(random.randint(1,6))
        content = r.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
        
        for link in soup.find_all('a', attrs={'class':'c-item_foot d-b t-l'}):
            links.append(link.get('href'))
    
    return links    

In [9]:
def clean_text (text):
    text = text.replace('\xa0', ' ')
    text = text.replace ('\n', ' ')
    text = text.replace ('  ', ' ')
    return text

In [10]:
def GetArticles(links):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    l = 'https://kanobu.ru'
    articles = []
    for link in links:
        art = KanobuArticle()
        page = requests.get(l+link, headers = headers)
        time.sleep(random.randint(1,3))
        content2 = page.content
        html2 = content2.decode('UTF-8')
        soup2 = BeautifulSoup(html2)
        
        date = soup2.find('div', attrs={'class':'c-title_footer'}).get_text()
        art.date = date.split(', ')[0]
        art.time = date.split(', ')[1]
        
        author = soup2.find('a', attrs={'class':'c-title_author'}).get_text()
        art.author = author
        
        text = soup2.find('div', attrs={'class':'c-detail_content'}).get_text()
        art.text = clean_text(text)
        
        words = len(clean_text(text).split())
        art.words = words
        
        title = soup2.find('h1', attrs={'class':'c-title_body'}).get_text()
        art.head = clean_text(title)
        
        rubric = soup2.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
        r = rubric.split(' | ')
        art.rubr = r[0]
        art.hashtag = r[1]
        #print(art)
        #print(text)
        #print(date)
        #print(author)
        print(title)
        print(words)
        #print(r[0])
        #print(r[1])
        #print(rubric)
        articles += [art]
    return articles

In [11]:
articles = GetArticles(GetPagesArticles(1))

В ремейке Resident Evil 3 вернутся неломающиеся ножи
164
Слух: Брент Спайнер появится заключительном эпизоде «Звездный путь: Пикар»
160
Новые эксклюзивы Epic Games Store: что появится в магазине весной
169
Топовые футболисты проведут благотворительный стрим по FIFA 20
158
Конор Макгрегор избил коронавирус. Спортсмен тренируется в подвале
170
«Плохие парни» выйдут в онлайн-кинотеатрах
168
Игровой сервис GeForce Now стал бесплатным на месяц
181
Шварценеггер ест из одной тарелки с пони и ослом. Это очень милое видео
193
Amazon публикует фильмы, которые раньше времени вышли в цифровой прокат
149
Опубликован первый скриншот S.T.A.L.K.E.R. 2. Его сделали на движке игры
252
13 логотипов известных брендов, которые изменил коронавирус
347
Российский блогер BadComedian рассказал о работе в кинотеатре сети «КАРО»
189
Мод позволяет раздеть Джилл в демо Resident Evil 3 Remake
150
Little Big все еще могут поучаствовать в «Евровидении 2020». Конкурс хотят провести в другом формате
207
Юрий Хованский 

### База

In [12]:
es=Elasticsearch()

In [13]:
es

<Elasticsearch([{}])>

In [14]:
es.indices.delete(index="kanobu") 

{'acknowledged': True}

In [15]:
es.indices.create(index="kanobu")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'kanobu'}

In [16]:
mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":" HH:mm"},
                                "words":{"type":"double"},
                                "hashtag":{"type":"text","analyzer" : "russian"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}

In [17]:
es.indices.put_mapping(index="kanobu", doc_type='article', body=mapit, include_type_name=True)

{'acknowledged': True}

In [18]:
es.indices.get_alias("*")

{'kanobu': {'aliases': {}}}

In [19]:
%%time
i = 0
for article in tqdm(articles):
    es.index(index="kanobu", doc_type='article', id = i, body=article.toDict())
    i+=1

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 22.85it/s]


Wall time: 864 ms


In [16]:
res = es.search(index="kanobu", body={"query": {"match_all": {}}})
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '0',
  '_score': 1.0,
  '_source': {'date': '22 марта 2020',
   'time': '15:14',
   'rubrics': 'Игры',
   'hashtag': '#россия',
   'title': 'В России заработал портал для тех, кто скучает на карантине. Это каталог бесплатных онлайн-услуг',
   'words': 183,
   'author': 'Анастасия Яблоновская',
   'text': '21 марта начал работу портал «Доступвсем» с информацией о компаниях и сервисах, которые предоставили бесплатный доступ к своим ресурсам. Сайт создан для тех, кто находится в изоляции из-за пандемии. Портал представляет собой каталог, где собраны акции и предложения от банков, онлайн-кинотеатров, операторов связи, сервисов доставки и образовательных площадок. «Школьники и студенты смогут продолжить занятия в виртуальных классах. А при желании — познакомиться с лекциями других преподавателей, самых лучших в стране. Стали доступнее дистанционные консультации врачей — по любой специальности. В открытом доступе есть русская и зарубежная 

### Добавление в базу новой статьи по ссылке

In [20]:
def add_article(link):
    articles = []
    art = KanobuArticle()
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    page = requests.get(link, headers = headers)
    content = page.content
    html = content.decode('UTF-8')
    soup = BeautifulSoup(html)
    
    date = soup.find('div', attrs={'class':'c-title_footer'}).get_text()
    art.date = date.split(', ')[0]
    art.time = date.split(', ')[1]
        
    author = soup.find('a', attrs={'class':'c-title_author'}).get_text()
    art.author = author
        
    text = soup.find('div', attrs={'class':'c-detail_content'}).get_text()
    art.text = clean_text(text)
        
    words = len(clean_text(text).split())
    art.words = words
        
    title = soup.find('h1', attrs={'class':'c-title_body'}).get_text()
    art.head = clean_text(title)
        
    rubric = soup.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
    r = rubric.split(' | ')
    art.rubr = r[0]
    art.hashtag = r[1]
    
    #print(art)
    #print(text)
    #print(date)
    #print(author)
    #print(title)
    #print(words)
    #print(r[0])
    #print(r[1])
    
    articles += [art]
    
    res = es.search(index="kanobu", body={"query": {"match_all": {}}})
    maxid = res['hits']['total']['value']
    
    for article in articles:
        es.index(index="kanobu", doc_type='article', id = maxid, body=article.toDict())
        
    print('Done')

In [21]:
link = 'https://kanobu.ru/news/entoni-hopkins-syigral-nafortepiano-dlya-svoego-kota-i-pokoril-sotsseti--422323/'
add_article(link)

19
Done


### Мэтч по id новой статьи

In [22]:
es.get(index="kanobu", doc_type='article', id=18)

{'_index': 'kanobu',
 '_type': 'article',
 '_id': '18',
 '_version': 1,
 '_seq_no': 18,
 '_primary_term': 1,
 'found': True,
 '_source': {'date': '20 марта 2020',
  'time': '12:59',
  'rubrics': 'Кино и сериалы',
  'hashtag': '#энтони хопкинс',
  'title': 'Энтони Хопкинс сыграл на фортепиано для своего кота и покорил соцсети ',
  'words': 163,
  'author': 'Анастасия Яблоновская',
  'text': 'Умиротворяющим видео поделился с подписчиками Энтони Хопкинс. Он сыграл на фортепиано для своего кота. Судя по ролику, питомец музыкальным талантом актера доволен. «Niblo заботится о том, чтобы я оставался здоровым, а взамен требует себя развлекать», — отметил Хопкинс в подписи к видео. Баста дал концерт в онлайне. Все, чтобы спасти нас от скуки Ролик получил большую в сети. На Facebook у него уже более 6 млн. просмотров. В комментариях поклонники актера назвали видео замечательным и очень милым. А еще пожелали кумиру беречь здоровье. Вчера вдохновляющим роликом поделилась с фанатами актриса Галь Га

### Мэтч по заголовку

In [78]:
res = es.search(index="kanobu", body={"query": {"match": {"title": "россия"}}})
res["hits"]["hits"]

[]

### Мэтч по дате

In [23]:
res = es.search(index="kanobu", body={"query": {"match": {"date": "22 марта 2020"}}})
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '0',
  '_score': 0.6040209,
  '_source': {'date': '22 марта 2020',
   'time': '15:14',
   'rubrics': 'Игры',
   'hashtag': '#россия',
   'title': 'В России заработал портал для тех, кто скучает на карантине. Это каталог бесплатных онлайн-услуг',
   'words': 183,
   'author': 'Анастасия Яблоновская',
   'text': '21 марта начал работу портал «Доступвсем» с информацией о компаниях и сервисах, которые предоставили бесплатный доступ к своим ресурсам. Сайт создан для тех, кто находится в изоляции из-за пандемии. Портал представляет собой каталог, где собраны акции и предложения от банков, онлайн-кинотеатров, операторов связи, сервисов доставки и образовательных площадок. «Школьники и студенты смогут продолжить занятия в виртуальных классах. А при желании — познакомиться с лекциями других преподавателей, самых лучших в стране. Стали доступнее дистанционные консультации врачей — по любой специальности. В открытом доступе есть русская и заруб

### Виджеты

In [53]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

"Требования к нереляционным БД
2 - нереляционная БД
2 - красивая структура БД
2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)
2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)
2 - зависит от БД: Redis - использование ключей, хешей, ...; Neo4j - нахождение путей не только к соседним вершинам, операции с графами; MongoDB, ElasticSearch - работа с текстами или географией; остальные БД - будем договариваться."

### 1 показ всех статей

In [129]:
button1 = widgets.Button(description='Match all articles') 
out = widgets.Output()

def on_button_clicked(_):
    with out:
        clear_output()
        res = es.search(index="kanobu", body={"query": {"match_all": {}}})
        for part in res["hits"]["hits"]:
            print(part + '\n')

In [130]:
button2 = widgets.Button(description='Match with options')
out = widgets.Output()

def on_button_clicked2(_):
    with out:
        clear_output()
        res = es.search(index="kanobu", body={"query": {"match": {menu.value : text.value}}})
        for part in res["hits"]["hits"]:
            print(part)

In [131]:
button1.on_click(on_button_clicked)
button2.on_click(on_button_clicked2)

In [132]:
text1 = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [133]:
menu = widgets.Dropdown(
       options=['id', 'title', 'date', 'author', 'hashtag', 'rubrics'],
       value='id',
       description='How to find?')

In [134]:
text = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [135]:
box = widgets.VBox([button1, menu, text, button2, out])
box

- добавить удаление
- поиск по Id выполняется по другому (придумать как добавить его в ту же кнопку)
- кнопка для добавления статьи по ссылке
- ... 